In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/Vivo_new.csv')

In [3]:
df.duplicated().sum()

12

In [4]:
df_V = df.drop_duplicates()
df_V.duplicated().sum()
df_V.shape

(193, 5)

In [5]:
df_V.dtypes

Company Name      object
Ratings           object
Specifications    object
Mobile Price      object
offer             object
dtype: object

In [6]:
#mobile brand name
pd.options.mode.chained_assignment = None  # default='warn'
df_V['Name']=df_V['Company Name'].str.extract('(vivo)')

In [7]:
df_V['test'] = df_V['Company Name'].str.extract('(\([^)]*\,)')
df_V['test1']=df_V['test'].astype(str)
df_V['Color'] = df_V['test1'].apply(lambda x: x.replace('(','').replace(',','').replace('nan','Not Mentioned'))

In [8]:
df_V['test2'] = df_V['Company Name'].str.split('(').str[0]
df_V['Model']= df_V['test2'].str.split().str[1:].str.join(' ')

In [9]:
def match(x):
    if re.search("Ratings",x):
        return x
    else:
        return "0"

In [10]:
df_V['test3']=df_V['Ratings'].apply(match)
df_V['test5']=df_V['test3'].apply(lambda x: x[x.find("vivo") : x.find("Ratings")])
df_V['Ratings(*)']=df_V['test3'].str.extract('(4\.\d|3\.\d)')

In [11]:
df_V['test4']=df_V['test3'].apply(lambda x: x[x.find(")")+1:x.find("Ratings")])

In [12]:
df_V['Ratings(*)']=df_V['test4'].str.extract('(4\.\d|3\.\d)')
df_V['Ratings(*)']=df_V['Ratings(*)'].fillna(0)

In [13]:
df_V['Reviews']=df_V['test4'].str[3:]
df_V['MobilesSold']=df_V['Reviews'].str.replace(',','')
df_V['MobilesSold']=df_V['MobilesSold'].replace('',0)
df_V['MobilesSold']=df_V['MobilesSold'].astype(int)

In [14]:
df_V['OfferInPerentage'] =df_V['offer'].str.extract('(\d+)')
df_V['OfferInPerentage']=df_V['OfferInPerentage'].fillna(0)
df_V['OfferInPerentage']=df_V['OfferInPerentage'].astype(int)

In [15]:
df_V['test5']=df_V['Mobile Price'].apply(lambda x: x.replace('%','').replace('offFree','').replace('delivery','').replace('off','').replace('Free delivery','').replace('Free',''))

In [16]:
df_V['test6']=df_V['test5'].str.extract('(\₹[^)]*\₹)')

In [17]:
df_V['DiscountedPrice']=df_V['test6'].apply(lambda x:str(x).replace('₹','').replace(',',''))
#Replacing NAN Values from Price(with discount) column
df_V['DiscountedPrice']=df_V['DiscountedPrice'].replace('nan','0')
df_V['DiscountedPrice']=df_V['DiscountedPrice'].astype(int)

In [18]:
df_V['MaxRetailPrice'] = df_V.DiscountedPrice + (df_V.OfferInPerentage/100 * df_V.DiscountedPrice)

In [19]:
df_V['Sales(₹ CR)']= round((df_V.DiscountedPrice*df_V.MobilesSold)/10000000,2)

In [20]:
df_V['test7']= df_V['Specifications'].str.split('|'and'ROM').str[0]
df_V[['RAM(GB)','ROM(GB)']]= df_V['test7'].str.split('|',expand = True) 
df_V['RAM(GB)']=df_V['RAM(GB)'].apply(lambda x: str(x).replace('GB RAM',''))
df_V['ROM(GB)']=df_V['ROM(GB)'].apply(lambda x: str(x).replace('GB',''))

In [21]:
df_V['Display']= df_V['Specifications'].str.extract('\((.*?)\)')
df_V['Display']=df_V['Display'].apply(lambda x: str(x).replace('inch','').replace('Inch',''))

In [22]:
df_V['Screen']=df_V['Specifications'].str.extract('(Full HD|HD|Full HD+ AMOLED)')
df_V['Screen']=df_V['Screen'].fillna("Not Mentioned")

In [23]:
df_V['test8']= df_V['Specifications'].apply(lambda x: x[x.find("Display"):x.find("Front")])

In [24]:
df_V['test9'] = df_V['test8'].apply(lambda x:str(x).replace('Display','').replace('+',''))

In [25]:
df_V[['Rear Camera','Front Camera']]= df_V['test9'].str.split('|',expand = True)

In [26]:
df_V['No of Rear Camera']= df_V['Rear Camera'].str.count("MP")
df_V['No of Front Camera']= df_V['Front Camera'].str.count("MP")
df_V['No of Front Camera']= df_V['No of Front Camera'].fillna(0)

In [27]:
df_V['Battery']= df_V['Specifications'].str.extract('(\d{4} mAh)')
df_V['Battery']=df_V['Battery'].apply(lambda x: str(x).replace('mAh',''))

In [28]:
df_V['Processor']= df_V['Specifications'].str.extract('(Snapdragon 695|Dimensity 6020|Mediatek Dimensity 7200|Helio P35|Helio G85|Mediatek Dimensity 1200)')
df_V['Processor']=df_V['Processor'].fillna("Not Mentioned")

In [29]:
# drop all test columns and unwanted columns
df_V.drop(columns=['test','test1','test2','test3','test4','test5','test6','test7','test8','test9','Company Name','Ratings','Specifications','Mobile Price','offer'],inplace=True,axis=1)

In [30]:
#df_V.rename(columns = {'prrice':'Price'}, inplace = True)
#df_V.head(5)
#df_V.iloc[0:0]
#df_V.dtypes
#df_V.drop(columns=['Sales(₹ CR)'],axis=1,inplace=True)
df_V.to_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/VivoCleaned.csv',index=False)